### 4.1 Four branches of machine learning
- Supervised learning监督学习，最常见的形式。包括sequence generation、object detection、image segmentation等最新应用。
- Unsupervised learning无监督学习，其目的在于数据可视化、数据压缩、去噪或更好的理解数据中的相关性。最广泛的无监督学习方法包括了dimensionality reduction与clustering。
- Self-supervised learning自监督学习，是监督学习的一个特例。指的是没有人工标注的lable的监督学习。lables是从输入数据中使用启发式算法生成的。最well-known的例子包括了autoencoder、时序监督学习中给定视频中过去的帧预测下一帧、给定文本中前面的词预测下一个单词等。
- Reinforcement learning强化学习，由agent接收有关其环境的信息，并学会选择使某种奖励最大化的行动。

### 4.2 Evaluating machine-learning models

回答一个问题：为何一定需要训练集、验证集、测试集而不能根据测试集的反馈结果调整模型的训练过程？
- 在验证集上根据metrics来调整hyperparameters这一过程本质上也是一种学习：在某个参数空间中寻找良好的模型配置。因此模型仍然会在验证集上过拟合，即验证集发生了information leak，即使你并没有在验证集的数据上训练模型也是如此。

解决这一问题有以下几种可行方法：
- hold-out validation最简单的留出验证，留出一定比例的训练数据作为验证集，在训练数据上训练模型并在验证数据上评估模型。调节好hyperparameters后在全体训练数据上重新训练出最终模型。这一方法的缺点在于，如果可用数据很少，可能验证集和训练集包含的样本过少，无法在统计学上代表数据，在选择验证集时进行随机打乱后训练出的模型performance差别会比较大。

- K-fold validation K折交叉验证。在ch3中回归问题中已经介绍过。

- iterated K-fold validation with shuffling 带有打乱数据的重复K折验证。即多次使用K折验证，在每次将数据划分为K个分区前都打乱一次数据，最终score是每次K folds的平均值。一共要训练并评估PxK个模型，计算代价较大但在Kaggle竞赛中很有用。

### 4.3 Data preprocessing, feature engineering, and feature learning

#### 4.3.1 Data preprocessing for neural networks

对于数据的预处理分为以下三步：
1. Data vectorization，把输入的声音、图像、文本等数据和目标数据转换为浮点数张量的形式。

2. Value Normalization 如果不做normalization，不同feature之间会有bias，且较大的梯度更新可能导致网络无法收敛。因此一般会将feature标准化到标准正态分布。对于Numpy数组可以实现如下：x -= mean(axis = 0) x/= x.std(axis=0)

3. Handling Missing values 如果数据中有missing value，可以简单的将值设置为0，因为神经网络可以学到0意味着缺失数据，并且会忽略这个值。

#### 4.3.2 Feature engineering

特征工程(feature engineering)是指将数据输入模型之前，利用你自己关于数据和机器学习算法的知识对于数据进行硬编码的变换(不是模型学到的)，以改善模型的学习效果。

一个很简单的例子：给的数据是时钟的照片，你抽取了时钟指针坐标作为feature。但是你知道时钟用角度表示才是更好的representation，这时候你可以硬编码写一个(x,y)坐标到极坐标的映射来得到更好的特征。而这就是特征工程的本质：**用更简单的方式表述问题，从而使问题变得更容易。**

对于现代深度学习，大部分特征工程并不是必要的，因为神经网络能够自己从数据中学习更好的表示。但特征工程仍然有其价值：
- 良好的特征仍然可以让你用更少的资源更优雅的解决问题
- 良好的特征可以让你用更少的数据解决问题

### 4.4 Overfitting and underfitting

虽然已经基本了解了overfitting、underfitting以及对应的拟合方法、regularization方法。但书中的一些提法还是令我有耳目一新的感觉：

- 机器学习的根本问题是优化optimization和泛化generalization的对立。机器学习的目的当然是得到良好的泛化，但泛化无法控制，只能基于训练数据调节模型。
- 训练开始时，优化和泛化是相关的，这时模型是underfit的。但在训练数据上迭代一定次数后，泛化不再提高并随着模型开始学习仅和训练数据有关的模式下降。
- 对overfitting最优解决方法是获取更多的训练数据。次优解决方法是调节模型允许存储的信息量，或对模型允许存储的信息加以约束。如果一个模型只能学习到几种模式，那么regularization能够迫使模型学习最重要的模式，得到泛化能力更好的model。

#### 4.4.1 Reducing the network's size

模型的容量capacity即模型中可学习参数的个数。参数更多的模型有着更大的记忆容量，更容易使得模型有能力学会训练样本和目标之间的字典式映射。也就是模型只是“死记硬背”了答案，而没有学到任何可泛化的知识。

故减少网络的capacity也是防止overfitting的一个方法。但capacity不足也可能欠拟合，因此要在容量过大和容量不足中找到一个折中。

#### 4.4.2 Adding weight regularization

提到regularization，人们总会谈到奥卡姆剃刀原理：如果一件事情有两种解释，那么最可能正确的解释就是最简单的那个，即假设更少的那个。曾教授过我的物理老师们也深受此影响，总会说：物理规律的形式是简单的，因为大自然就遵循着这种简单美。

在神经网络中也是如此，我们会认为简单的模型比复杂的模型更不容易过拟合，或者说有着更强的泛化能力。

这里的simple model指的是**参数值分布的熵更小的模型**，因此，一种方法是强制让模型的weights只能取较小的值，从而限制模型的复杂度，这也就是weight regularization。具体实现方法是向loss的函数中增加较大权重的cost：
- L1 regularization cost与weights绝对值成正比
- L2 regularization cost与weights的平方成正比，L2 正则化也叫weight decay

在Keras中，添加权重正则化的方法是在层中传递regularizers instance：

In [3]:
from keras import regularizers
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                      activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                      activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

C:\Users\10750\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


需要注意的是，weight regularization中的惩罚项**只在训练时添加**，所以训练的loss会比test时大很多。

#### 4.4.3 dropout regularization

dropout是神经网络最有效也最常用的正则化方法之一。即根据某一固定的dropout rate，在输出时随机将这一层的activation输出结果中的某些特征设置为0。

注意：实践中我们一般dropout之后再将activation layer根据dropout rate成比例放大，测试时就不需要dropout了。

Hinton在发明dropout时的motivation是银行的防欺诈机制，其核心思想是在层的输出值中引入noise，打破不显著的偶然模式。

In [4]:
model.add(layers.Dropout(0.5))

以上语句在model中加入了dropout层，其将会被应用在前一层layer的输出(activation)。

### 4.5 The universal workflow of machine learning

1. Defining the problem and assembling a dataset
2. Choosing a measure of success
3. Deciding on an evaluation protocol
4. Preparing your data
5. Developing a model that does better than a baseline
6. Scaling up: developing a model that overfits
7. Regularizing your model and tuning your hyperparameters